<a href="https://colab.research.google.com/github/Timochik/Hw_datascience/blob/main/Data_10_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [2]:
# Завантажимо датасет Fashion MNIST
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# Перетворимо дані у формат, який підходить для VGG16
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [9]:
# Щоб використати VGG16, треба змінити розмір зображень до 32x32 або 48x48, бо мінімальний розмір VGG16 - 32x32
X_train = tf.image.resize(X_train, (48, 48))
X_test = tf.image.resize(X_test, (48, 48))

In [10]:
# Нормалізуємо дані
X_train = tf.cast(X_train, tf.float32) / 255.0
X_test = tf.cast(X_test, tf.float32) / 255.0

In [11]:
# Завантажимо попередньо навчену модель VGG16 без верхніх шарів
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
# Заморозимо всі шари VGG16
vgg16_base.trainable = False

In [13]:
# Побудуємо нову модель
model = models.Sequential([
    layers.Input(shape=(48, 48, 1)),

    # Повторимо канали до 3 для сумісності з VGG16
    layers.Conv2D(3, (3, 3), padding='same', activation='relu'),

    # Додамо базу VGG16
    vgg16_base,

    # Додамо наші власні шари для класифікації
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

In [14]:
# Скомпілюємо модель
model.compile(optimizer=Adam(learning_rate=0.0001),  # Мала швидкість навчання
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [16]:
# Навчання моделі
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
   2/1500 ━━━━━━━━━━━━━━━━━━━━ 1:28:11 4s/step - accuracy: 0.0859 - loss: 2.3411


KeyboardInterrupt



In [ ]:
# Оцінка моделі
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\nТочність на тестових даних:', test_acc)

In [ ]:
# Візуалізація результатів
plt.plot(history.history['accuracy'], label='Точність на тренувальних даних')
plt.plot(history.history['val_accuracy'], label = 'Точність на валідаційних даних')
plt.xlabel('Епоха')
plt.ylabel('Точність')
plt.ylim([0.8, 1])
plt.legend(loc='lower right')
plt.show()
